# Expected Goal Model using logistic regression

In [2]:
import os
import pandas as pd
import numpy as np

import math

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import pyodbc
from sqlalchemy import create_engine


In [3]:
WINDOWS_HOST_IP = '172.24.176.1'
SERVER_PORT = 1433


query = """
WITH TAGS AS (
    SELECT 
        ET.eventRecordID,

        -- Flag cho các loại tag:
        MAX(CASE WHEN TN.Description = 'Goal' THEN 1 END) AS Goal,
        MAX(CASE WHEN TN.Description = 'Own goal' THEN 1 END) AS OwnGoal,
        MAX(CASE WHEN TN.Description = 'Counter attack' THEN 1 END) AS CounterAttack,

        -- Body part: lấy tagID (401,402,403) của cú dứt điểm
        MAX(CASE WHEN ET.tagID IN (401, 402, 403) THEN ET.tagID END) AS BodyPartTagID
    FROM EVENTTAGS ET
    LEFT JOIN TAGSNAME TN
        ON ET.tagID = TN.tagID
    GROUP BY ET.eventRecordID
)

SELECT
    EV.matchID,
    EV.matchPeriod,
    EV.eventSec,
    EN.eventName,
    EN.subEventName,
    EV.teamID,
    EV.posOrigX,
    EV.posOrigY,
    EV.posDestX,
    EV.posDestY,

    EV.playerID,
    P.Sname AS playerName,
    P.Prole AS playerPosition,
    P.foot AS playerStrongFoot,

    EV.teamID AS teamPossession,
    MT_home.teamID AS homeTeamId,
    MT_away.teamID AS awayTeamId,

    -- Flags
    ISNULL(T.Goal, 0) AS Goal,
    ISNULL(T.OwnGoal, 0) AS OwnGoal,
    ISNULL(T.CounterAttack, 0) AS CounterAttack,

    -----------------------------------------
    -- Body Part (KHÔNG nhân dòng)
    -----------------------------------------
    CASE
        WHEN T.BodyPartTagID = 401 THEN 'leftFoot'
        WHEN T.BodyPartTagID = 402 THEN 'rightFoot'
        WHEN T.BodyPartTagID = 403 THEN 'head/body'
        ELSE 'Unknow'
    END AS bodyPartShot,

    -- Body part code cho xG
    CASE
        WHEN T.BodyPartTagID = 401 THEN 1
        WHEN T.BodyPartTagID = 403 THEN 2
        WHEN T.BodyPartTagID = 402 THEN 3 
        ELSE 0
    END AS bodyPartShotCode

FROM EVENTS AS EV

-- Event name
LEFT JOIN EVENTSNAME AS EN
    ON EV.subEventID = EN.subEventID

-- Player
LEFT JOIN PLAYERS AS P
    ON EV.playerID = P.playerID

-- Home / Away teams
LEFT JOIN MATCHTEAMS AS MT_home
    ON EV.matchID = MT_home.matchID AND MT_home.side = 'home'

LEFT JOIN MATCHTEAMS AS MT_away
    ON EV.matchID = MT_away.matchID AND MT_away.side = 'away'

LEFT JOIN TAGS AS T
    ON EV.eventRecordID = T.eventRecordID

ORDER BY EV.matchID, EV.eventSec;
"""


# Định dạng URI
# 'mssql+pyodbc://<UID>:<PWD>@<DSN>' hoặc
# 'mssql+pyodbc:///?odbc_connect=<CONNECTION_STRING_ĐƯỢC_URL_ENCODE>'

sql_uri = (
    f"mssql+pyodbc://WSL2_SQL_ServerPort_1433:Dat23012003"
    f"@{WINDOWS_HOST_IP}:{SERVER_PORT}/csdl_Soccer"
    f"?driver=ODBC+Driver+18+for+SQL+Server"
    f"&TrustServerCertificate=yes"
)

engine = create_engine(sql_uri)

# Kết nối và Truy vấn
try:
    # Sử dụng engine trực tiếp trong read_sql
    df_events = pd.read_sql(query, engine)
    print("Truy vấn thành công bằng SQLAlchemy!")
except Exception as e:
    print("Lỗi khi sử dụng SQLAlchemy:", e)

Truy vấn thành công bằng SQLAlchemy!


In [4]:
df_events.head()

,matchID,matchPeriod,eventSec,eventName,subEventName,teamID,posOrigX,posOrigY,posDestX,posDestY,...,playerPosition,playerStrongFoot,teamPossession,homeTeamId,awayTeamId,Goal,OwnGoal,CounterAttack,bodyPartShot,bodyPartShotCode
0,1694390,2H,0.814000,Pass,Simple pass,11944,49,48,49,48,...,Midfielder,right,11944,4418,11944,0,0,0,Unknow,0
1,1694390,2H,0.814000,Pass,Simple pass,11944,39,50,39,50,...,Forward,right,11944,4418,11944,0,0,0,Unknow,0
2,1694390,1H,1.255990,Pass,Simple pass,4418,50,48,47,50,...,Forward,left,4418,4418,11944,0,0,0,Unknow,0
3,1694390,1H,2.351908,Pass,Simple pass,4418,47,50,41,48,...,Forward,left,4418,4418,11944,0,0,0,Unknow,0
4,1694390,2H,2.677000,Pass,High pass,11944,69,14,31,86,...,Midfielder,right,11944,4418,11944,0,0,0,Unknow,0


In [5]:
df_shots = df_events[df_events['eventName'] == 'Shot'].copy()

In [6]:
df_shots.head()

,matchID,matchPeriod,eventSec,eventName,subEventName,teamID,posOrigX,posOrigY,posDestX,posDestY,...,playerPosition,playerStrongFoot,teamPossession,homeTeamId,awayTeamId,Goal,OwnGoal,CounterAttack,bodyPartShot,bodyPartShotCode
24,1694390,1H,31.226217,Shot,Shot,4418,91,29,0,0,...,Midfielder,left,4418,4418,11944,0,0,0,rightFoot,3
61,1694390,2H,100.604872,Shot,Shot,11944,100,100,20,67,...,Midfielder,right,11944,4418,11944,0,0,0,rightFoot,3
85,1694390,2H,130.592908,Shot,Shot,11944,100,100,0,0,...,Forward,right,11944,4418,11944,0,0,0,rightFoot,3
89,1694390,1H,143.119551,Shot,Shot,11944,71,29,100,100,...,Midfielder,right,11944,4418,11944,0,0,0,rightFoot,3
137,1694390,1H,219.576026,Shot,Shot,11944,96,57,100,100,...,Forward,right,11944,4418,11944,0,0,0,rightFoot,3


In [10]:
total_shots = len(df_shots)
print(f"Tổng số cú sút: {total_shots}")
print(f"Xác suất ghi bàn khi thực hiện cú sút: {df_shots['Goal'].mean()*100:.2f}%")

Tổng số cú sút: 43071
Xác suất ghi bàn khi thực hiện cú sút: 10.42%
